In [93]:
import copy
import csv

# preparing arrays with files
files_input_sell = ['MS-b1-sell.csv','MS-b2-sell.csv', 'MS-m1-sell.csv', 'MS-m2-sell.csv', 'MS-s1-sell.csv', 'MS-s2-sell.csv', 'MS-s3-sell.csv', 'MS-s4-sell.csv', 'MS-s5-sell.csv']

files_input_supply = ['MS-b1-supply.csv','MS-b2-supply.csv', 'MS-m1-supply.csv', 'MS-m2-supply.csv', 'MS-s1-supply.csv', 'MS-s2-supply.csv', 'MS-s3-supply.csv', 'MS-s4-supply.csv', 'MS-s5-supply.csv']

files_input_inventory = ['MS-b1-inventory.csv','MS-b2-inventory.csv', 'MS-m1-inventory.csv', 'MS-m2-inventory.csv', 'MS-s1-inventory.csv', 'MS-s2-inventory.csv', 'MS-s3-inventory.csv', 'MS-s4-inventory.csv', 'MS-s5-inventory.csv']

files_answer_daily = ['MS-b1-daily.csv','MS-b2-daily.csv', 'MS-m1-daily.csv', 'MS-m2-daily.csv', 'MS-s1-daily.csv', 'MS-s2-daily.csv', 'MS-s3-daily.csv', 'MS-s4-daily.csv', 'MS-s5-daily.csv']

files_answer_steal = ['MS-b1-steal.csv','MS-b2-steal.csv', 'MS-m1-steal.csv', 'MS-m2-steal.csv', 'MS-s1-steal.csv', 'MS-s2-steal.csv', 'MS-s3-steal.csv', 'MS-s4-steal.csv', 'MS-s5-steal.csv']

files_output_daily = ['MS-b1-daily_output.csv','MS-b2-daily_output.csv', 'MS-m1-daily_output.csv', 'MS-m2-daily_output.csv', 'MS-s1-daily_output.csv', 'MS-s2-daily_output.csv', 'MS-s3-daily_output.csv', 'MS-s4-daily_output.csv', 'MS-s5-daily_output.csv']

files_output_steal = ['MS-b1-steal_output.csv','MS-b2-steal_output.csv', 'MS-m1-steal_output.csv', 'MS-m2-steal_output.csv', 'MS-s1-steal_output.csv', 'MS-s2-steal_output.csv', 'MS-s3-steal_output.csv', 'MS-s4-steal_output.csv', 'MS-s5-steal_output.csv']


# main function
def check(num):

    state = files_input_sell[num][0:2] # remembering state
    
    # opening file with sells
    with open(files_input_sell[num], 'r') as f:
        reader = csv.reader(f)
        sell_data = list(reader)
    f.close()

    # opening file with supplies
    with open(files_input_supply[num], 'r') as f:
        reader = csv.reader(f)
        supply_data = list(reader)
    f.close()

    # opening file with inventory
    with open(files_input_inventory[num], 'r') as f:
        reader = csv.reader(f)
        inventory_data = list(reader)
    f.close()

    # making list with sells in convenient form
    list_sell = [] # empty list
    i = 0
    last_date = sell_data[1][0]
    list_sell.append([last_date, 0, 0]) # appending first row
    sell_data = sell_data[1:]           # shift on one position in list with selling data
    for row in sell_data:
        if row[0] == last_date:
            if row[1][6:8] == 'ap':      # means that one apple has been sold
                list_sell[i][1] += int(1)
            else:                        # means that one pen has been sold           
                list_sell[i][2] += int(1)
        else:
            last_date = row[0] # changing sell day
            i += 1             # increasing number in sell list
            if row[1][6:8] == 'ap':            
                list_sell.append([last_date, 1, 0])  # means that one apple has been sold 
            else:
                list_sell.append([last_date, 0, 1])  # means that one pen has been sold


    # creating template for stolen items
    list_steal = []
    list_steal.append(['date', 'apple', 'pen'])

    # filling list about daily sells
    list_daily = copy.deepcopy(list_sell)
    k = 1   # counter in the inventory list
    i = 1   # control out of range error in list with supplies
    j = 0   # counter in the daily list
    m = 1   # control data in list with stolen goods

    list_sells_monthly = [] # count of sells per month
    cnt_apples = cnt_pens = 0
    list_steal_monthly = [] # count of steals per month

    for row in list_sell:
        if j == 0:                # filling zero element
            list_daily[j][1] = 0
            list_daily[j][2] = 0  
        else:                     # else assigns previous day values
            list_daily[j][1] = list_daily[j-1][1]  # for apples
            list_daily[j][2] = list_daily[j-1][2]  # for pens

        if ((i < len(supply_data)) and (row[0] == supply_data[i][0])):   #if it was a supply day 
            list_daily[j][0] = row[0]        
            list_daily[j][1] += (int(supply_data[i][1]) - row[1])   # adding supplies for apples     
            list_daily[j][2] += (int(supply_data[i][2]) - row[2])   # adding supplies for pens
            i += 1   
        else:
            list_daily[j][1] -=  int(row[1])  # decreasing number of apples
            list_daily[j][2] -=  int(row[2])  # decreasinf number od pens

            cnt_apples += int(row[1])
            cnt_pens += int(row[2])

        # dealing with stolen goods
        if (list_daily[j][0] == inventory_data[k][0]):                  # end of the month
            apples = str(list_daily[j][1] - int(inventory_data[k][1]))  # stolen apples
            pens = str(list_daily[j][2] - int(inventory_data[k][2]))    # stolen pens
            if (apples != '0') or (pens != '0'):                        # some goods have been stolen
                list_steal.append(['', '', ''])                         
                list_steal[m][1] = apples
                list_steal[m][2] = pens
                list_steal[m][0] = list_daily[j][0]                     # date
                m += 1

            list_daily[j][1] = int(inventory_data[k][1])                # putting data from inventory
            list_daily[j][2] = int(inventory_data[k][2])
            k += 1

            list_sells_monthly.append([list_daily[j][0], cnt_apples, cnt_pens])
            list_steal_monthly.append([list_daily[j][0], apples, pens])
            cnt_pens = cnt_apples = 0

        j += 1

    list_daily.insert(0, ['date', 'apple', 'pen'])                      # inserting captions
    
    # prepare and write worst and most successful months
    def sort_successfull_apples(inp):
        return inp[1]
    def sort_successfull_pens(inp):
        return inp[2]
    def sort_worst_apples(inp):
        return -inp[1]
    def sort_worst_pens(inp):
        return -inp[2]

    print("Most successull months by apples:")
    list_sells_monthly.sort(key = sort_successfull_apples)
    k = 0
    for row in list_sells_monthly:
        if k == 3:
            break
        print(row[0])

    print("Most successull months by pens:")
    list_sells_monthly.sort(key = sort_successfull_pens)
    k = 0
    for row in list_sells_monthly:
        if k == 3:
            break
        print(row[0])

    print("Worst months by apples:")
    list_sells_monthly.sort(key = sort_worst_apples)
    k = 0
    for row in list_sells_monthly:
        if k == 3:
            break
        print(row[0])
    
    print("Worst months by pens:")
    list_sells_monthly.sort(key = sort_worst_pens)
    k = 0
    for row in list_sells_monthly:
        if k == 3:
            break
        print(row[0])
    
    #checking daily
    with open(files_answer_daily[num]) as f:
        reader = csv.reader(f)
        check_daily = list(reader)
    f.close()

    temp_str = files_input_sell[num].replace('-sell.csv', '')
    i = 0    # for avoiding row with signatures
    k = 0    # for mistakes printing
    
    for row in check_daily: 
        if (i != 0):
            if ((row[0] != list_daily[i][0]) or (int(row[1]) != list_daily[i][1]) or (int(row[2]) != list_daily[i][2])): 
                print('Supposed to be ', row, ' in daily', temp_str)
                print('But we have ', list_daily[i], ' !')
                k = 1
                break
        i += 1
    if (k == 0):
        print('Correct information in daily list ', temp_str)    


    #checking stolen
    with open(files_answer_steal[num]) as f:
        reader = csv.reader(f)
        check_steal = list(reader)
    f.close()

    i = 0  # counter for information in steal list
    k = 0  # for mistakes printing

    for row in check_steal:       
        if (row != list_steal[i]):
            print('Supposed to be ', row, ' in steal', temp_str)
            print('But we have ', list_steal[i], ' !')
            k = 1
            break
        i += 1    
    if (k == 0):
        print('Correct information in steal list ', temp_str)

    # writing daily
    with open(files_output_daily[num], mode = 'w') as file:
        writer = csv.writer(file, delimiter = ',')
        for row in list_daily:
            writer.writerow(row)
    file.close()      

    # writing steal
    with open(files_output_steal[num], mode = 'w') as file:
        writer = csv.writer(file, delimiter = ',')
        for row in list_steal:
            writer.writerow(row)
    file.close()    
    
    
    list_result = [] #setuping grouped list
    list_result.append([list_sell[0][0][0:4], state, 0, 0, 0, 0])
    i = 0
    
    #grouping up "sold" and "stolen" lists
        
    #filling list with "sold" info
    for row in list_sell:
        if row[0][0:4] == list_result[i][0]: #same year
            list_result[i][2] += row[1]      # apples
            list_result[i][4] += row[2]      # pens
        else: #new year
            i += 1
            list_result.append(['', '', 0, 0, 0, 0]) #adding a row to our list
            list_result[i][1] = state
            list_result[i][0] = row[0][0:4]
            list_result[i][2] += row[1]
            list_result[i][4] += row[2]
  
    i = 0
    
    #same as above but for stolen
    for row in list_steal:
        if row[0] != 'date':
            if row[0][0:4] == list_result[i][0]: 
                list_result[i][3] += int(row[1])
                list_result[i][5] += int(row[2])
            else:
                i += 1
                list_result[i][0] = row[0][0:4]
                list_result[i][3] += int(row[1])
                list_result[i][5] += int(row[2])
    return list_result

 
list_all = []

# making daily and steal lists for all files, and group them
for i in range(0, len(files_input_sell)):
    list_all += check(i)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3652 non-null   object
dtypes: object(1)
memory usage: 28.7+ KB


In [ ]:
#sorting result list by state, then by year    
    
def sort_by_state(inp):
    return inp[1]
def sort_by_year(inp):
    return inp[0]

list_all.sort(key = sort_by_state)
list_all.sort(key = sort_by_year)

last_year = list_all[0][0]

list_pos_start = [0]
list_pos_delete = []

j = 0

#aggregating our list into rows, 1 row for 1 year and state

#collecting positions of new years(we need to keep them), 
#and positions of all other rows(we need to delete them, after summing all up)

for row in list_all:
    if (row[0] == last_year) and not(j in list_pos_start): 
        list_all[list_pos_start[-1]][2] += row[2] #summing rows
        list_all[list_pos_start[-1]][3] += row[3]
        list_all[list_pos_start[-1]][4] += row[4]
        list_all[list_pos_start[-1]][5] += row[5]
        list_pos_delete.append(j)
    if (row[0] != last_year):
        last_year = row[0]
        list_pos_start.append(j)
    j += 1
    

j = 0
#deleting not needed rows

for i in list_pos_delete:
    del list_all[i - j]
    j += 1
list_all.insert(0, ['year', 'state', 'apple_sold', 'apple_stolen', 'pen_sold', 'pen_stolen'])

# writing agregated table
with open('agregate_output.csv', mode = 'w') as file:
        writer = csv.writer(file, delimiter = ',')
        for row in list_all:
            writer.writerow(row)
file.close()